# Lesen und Visualisieren von Rohdaten, Zuschneiden, Filtern und Speichern

## Importiere die wenigen Pakete, die ich brauche

In [117]:
import matplotlib.pyplot as plt
import mne 
import os
import numpy as np 
from pathlib import Path
from mpl_toolkits.mplot3d import Axes3D  

Matplotlib das Qt5Agg-Backend verwendet, das die beste Wahl für die interaktiven Plotfunktionen von Python ist.

In [118]:
matplotlib.use('Qt5Agg')

In [119]:
sample_data_folder = r"C:\Users\KARMERY\Desktop\tau"

#  In einen pathlib.Path konvertieren für effizienter zu sein. 

sample_data_folder = pathlib.Path(sample_data_folder)
sample_data_folder

WindowsPath('C:/Users/KARMERY/Desktop/tau')

## Noch einige Rohdaten laden.

In [120]:
raw_path = sample_data_folder / 'recep.ahdr'
raw = mne.io.read_raw(raw_path)
raw

Extracting parameters from C:\Users\KARMERY\Desktop\tau\recep.ahdr...
Setting channel info structure...


C:\Users\KARMERY\AppData\Local\Temp\ipykernel_15244\3603324103.py:2: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw(raw_path)


<RawBrainVision | recep.eeg, 17 x 1145856 (1145.9 s), ~22 kB, data not loaded>

## Visualisierung der Rohdaten!

In [122]:
raw.plot()

<MNEBrowseFigure size 800x800 with 4 Axes>

Channels marked as bad:
none


## Ereignisse aus den `STIM`-Kanälen extrahieren

In [123]:
events = mne.events_from_annotations

In [124]:
event_id = {
    'sol-1': 99999,
    'sag-0.5': 10025,
    'sol-1': 10024,
}
event_id

{'sol-1': 10024, 'sag-0.5': 10025}

## Die Rohdaten erneut zeichnen, aber die Ereignismarkierungen hinzufügen.

## Informationen über den Daten Sammeln

In [125]:
raw.info

<Info | 7 non-empty values
 bads: []
 ch_names: FP1, FP2, FC5, F3, Fz, F4, FC6, C3, Cz, C4, P3, Pz, P4, PO3, ...
 chs: 17 EEG
 custom_ref_applied: False
 highpass: 0.1 Hz
 lowpass: 70.0 Hz
 meas_date: 2023-03-03 11:44:25 UTC
 nchan: 17
 projs: []
 sfreq: 1000.0 Hz
>

In [126]:
raw.info['meas_date']

datetime.datetime(2023, 3, 3, 11, 44, 25, 769142, tzinfo=datetime.timezone.utc)

In [127]:
raw.info['sfreq']

1000.0

In [128]:
raw.info['bads']

[]

In [129]:
raw.ch_names[:17]

['FP1',
 'FP2',
 'FC5',
 'F3',
 'Fz',
 'F4',
 'FC6',
 'C3',
 'Cz',
 'C4',
 'P3',
 'Pz',
 'P4',
 'PO3',
 'PO4',
 'Oz',
 'gsr']

In [130]:
raw.info['chs'][0]

{'ch_name': 'FP1',
 'coil_type': 1 (FIFFV_COIL_EEG),
 'kind': 2 (FIFFV_EEG_CH),
 'logno': 1,
 'scanno': 1,
 'cal': 0.0488281,
 'range': 1e-06,
 'loc': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 'unit': 107 (FIFF_UNIT_V),
 'unit_mul': 0 (FIFF_UNITM_NONE),
 'coord_frame': 4 (FIFFV_COORD_HEAD)}

## Visualisierung die Sensorpositionen 

In [131]:
mne.channels.make_standard_montage

<function mne.channels.montage.make_standard_montage(kind, head_size='auto')>

In [132]:
 mne.channels.DigMontage(dig=None, ch_names=None)

<DigMontage | 0 extras (headshape), 0 HPIs, 0 fiducials, 0 channels>

In [133]:
dig : {"one": 1, "two": 2, "three": 3}

In [134]:
ch_names :['FP1','FP2','FC5','F3','Fz','F4','FC6','C3','Cz','C4','P3','Pz','P4','PO3','PO4','Oz','gsr']

In [135]:
mne.channels.get_builtin_montages()

builtin_montages = mne.channels.get_builtin_montages(descriptions=True)
for montage_name, montage_description in builtin_montages:
    print(f'{montage_name}: {montage_description}') 


standard_1005: Electrodes are named and positioned according to the international 10-05 system (343+3 locations)
standard_1020: Electrodes are named and positioned according to the international 10-20 system (94+3 locations)
standard_alphabetic: Electrodes are named with LETTER-NUMBER combinations (A1, B2, F4, …) (65+3 locations)
standard_postfixed: Electrodes are named according to the international 10-20 system using postfixes for intermediate positions (100+3 locations)
standard_prefixed: Electrodes are named according to the international 10-20 system using prefixes for intermediate positions (74+3 locations)
standard_primed: Electrodes are named according to the international 10-20 system using prime marks (' and '') for intermediate positions (100+3 locations)
biosemi16: BioSemi cap with 16 electrodes (16+3 locations)
biosemi32: BioSemi cap with 32 electrodes (32+3 locations)
biosemi64: BioSemi cap with 64 electrodes (64+3 locations)
biosemi128: BioSemi cap with 128 electrodes (1

In [136]:
mne.Info
mne.viz.plot_sensors

<function mne.viz.utils.plot_sensors(info, kind='topomap', ch_type=None, title=None, show_names=False, ch_groups=None, to_sphere=True, axes=None, block=False, show=True, sphere=None, pointsize=None, linewidth=2, verbose=None)>

In [137]:
mne.datasets.ssvep.data_path

<function mne.datasets.ssvep.ssvep.data_path(path=None, force_update=False, update_path=True, download=True, *, verbose=None)>

In [138]:
ssvep_folder = mne.datasets.ssvep.data_path()  
ssvep_data_raw_path = (ssvep_folder / 'sub-02' / 'ses-01' / 'eeg' /
                       'sub-02_ses-01_task-ssvep_eeg.vhdr')
ssvep_raw = mne.io.read_raw_brainvision(ssvep_data_raw_path, verbose=False)

In [139]:
# Für manuelle Kanalen zu importieren
ssvep_raw.set_montage(easycap_montage)
fig = ssvep_raw.plot_sensors(show_names=True)

## Kanäle als schlecht markieren

Ich markiere einen zusätzlichen EEG-Kanal als schlecht und sehe mich das Topoplot an.

In [141]:
raw.info['bads'] 

[]

In [163]:
raw.info['bads'] = ['Fz'] 
ssvep_raw.plot_sensors(ch_type='eeg')

<Figure size 640x640 with 1 Axes>

In [167]:
fig = plt.figure()
ax2d = fig.add_subplot(121)
ax3d = fig.add_subplot(122, projection='3d')
sample_raw.plot_sensors(ch_type='eeg', axes=ax2d)
sample_raw.plot_sensors(ch_type='eeg', axes=ax3d, kind='3d')

<Figure size 640x480 with 2 Axes>

## Nur eine Teilmenge der Kanäle auswählen.

In [168]:
raw_eeg = raw.copy().pick_types(meg=False, eeg=True, eog=False, exclude=[])
len(raw_eeg.ch_names)

17

In [169]:
raw_eeg.info

<Info | 8 non-empty values
 bads: 1 items (Fz)
 ch_names: FP1, FP2, FC5, F3, Fz, F4, FC6, C3, Cz, C4, P3, Pz, P4, PO3, ...
 chs: 17 EEG
 custom_ref_applied: False
 highpass: 0.1 Hz
 lowpass: 70.0 Hz
 meas_date: 2023-03-03 11:44:25 UTC
 nchan: 17
 projs: []
 sfreq: 1000.0 Hz
>

## Crop die Daten und filtern.

In [ ]:
raw_eeg_cropped = raw_eeg.copy().crop(tmax=100)
raw_eeg_cropped.times[-1]

In [171]:
raw_eeg_cropped.load_data()
raw_eeg_cropped_filtered = raw_eeg_cropped.copy().filter(l_freq=0.1, h_freq=40)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 33001 samples (33.001 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    0.7s finished


In [172]:
raw_eeg_cropped.plot(events=events, event_id=event_id, title='Unfiltered' )
raw_eeg_cropped_filtered.plot(events=events, event_id=event_id, title='Filtered')

TypeError: 'function' object is not subscriptable

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2)

raw_eeg_cropped.plot_psd(ax=ax[0], show=False)
raw_eeg_cropped_filtered.plot_psd(ax=ax[1], show=False)

ax[0].set_title('PSD vor dem Filtern')
ax[1].set_title('PSD nach dem Filtern')
ax[1].set_xlabel('Frequenz (Hz)')
fig.set_tight_layout(True)
plt.show()

## Speichern die Daten!

In [173]:
raw_eeg_cropped_filtered.save(pathlib.Path('out_data') / 'eeg_cropped_filt_raw.fif', 
                              overwrite=True)

Writing C:\Users\KARMERY\Desktop\MNE\pybrain_mne-main\out_data\eeg_cropped_filt_raw.fif
Closing C:\Users\KARMERY\Desktop\MNE\pybrain_mne-main\out_data\eeg_cropped_filt_raw.fif
[done]
